## 1 кейс

**Ваша задача написать функцию `process_files`, которая принимает на вход два пути к папкам. Из первой папки необходимо выбрать все "чеки" (файлы по шаблону из условия), а во вторую папку сохранить один объединенный чек (отсортированный по дате, а затем по продукту) под названием `combined_data.csv`.**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить папку с файлами. После выполнения, в папке `reports_main` будут храниться все присланные магазинами чеки.

In [ ]:
!wget https://github.com/vs8th/reports/archive/main.zip

import zipfile

with zipfile.ZipFile("main.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

!rm main.zip

--2025-03-09 15:02:24--  https://github.com/vs8th/reports/archive/main.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Vs8th/reports/zip/refs/heads/main [following]
--2025-03-09 15:02:24--  https://codeload.github.com/Vs8th/reports/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.121.10
Connecting to codeload.github.com (codeload.github.com)|140.82.121.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ]   3.22K  --.-KB/s    in 0s      

2025-03-09 15:02:24 (46.0 MB/s) - ‘main.zip’ saved [3296]



Чтобы посмотреть как выглядят подходящие для объединения чеки выполните следующую ячейку.

In [ ]:
import pandas as pd

df = pd.read_csv('reports-main/2023-02-17-05-38-2.csv', sep=";")
df

,date,product,store,cost
0,2023-02-17,product_0,store_2,10
1,2023-02-17,product_1,store_2,20
2,2023-02-17,product_2,store_2,30
3,2023-02-17,product_3,store_2,40
4,2023-02-17,product_4,store_2,50


**Решение**

**Примечание**

Не все файлы подходящие по названию, будут подходить по содержанию. Там может оказаться лишний столбец, например. Ориентируйтесь на столбцы из чека выше - это то, что вас интересует. Остальные столбцы можно просто отбросить.

**Важно**: разделителем файла на выходе должна быть запятая.

In [ ]:
# Импортируем необходимые библиотеки для работы с файлами
import os, re

In [ ]:
def file_name_match(file):
  '''
  Отдельная функция, которая проверяет имя файла по шаблону
  True - если соответствует, False - если не соответствует
  '''
  pattern = r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d+.csv'
  return bool(re.match(pattern, file))

In [ ]:
def process_files(src_folder, dest_folder):
  # Получим список фалов директории src_folder и сразу проверяем, что это именно
  # файл csv, и что имена файлов соответствуют шаблону
  files = [f for f in os.listdir(src_folder) \
           if os.path.isfile(os.path.join(src_folder, f)) \
           and file_name_match(f)]
  # Про сортировку не сказано в условии, но без неё проверка не проходит
  files.sort()
  # Теперь в files содержатся нужные нам имена файлов

  # Пройдёмся по всем файлам из files и запишем их строки в результирующий файл
  # Проверяем существует ли dest_folder и создаём её, если не существует
  os.makedirs(dest_folder, exist_ok=True)
  # Возбмём заголовки из первого файла
  with open(f'{src_folder}/{files[0]}', 'r') as f:
    cols = f.readline()[:-1].split(';')
  # Откроем конечный файл на запись, чтобы сразу записывать результаты
  with open(f'{dest_folder}/combined_data.csv','w') as output_f:
    output_f.write(','.join(cols) + '\n') # Сформируем заголовки через ","
    for f in files:
      with open(f'{src_folder}/{f}', 'r') as input_f:
        # Прочитаем заголовки в фалйе, считем, что заголовок всегда есть
        caps = input_f.readline()[:-1].split(';')
        # Запомним номера столбцов для cols
        # Здесь считаем, что такие столбцы всегда есть
        my_cols = [caps.index(cols[i]) for i in range(len(cols))]
        for line in input_f:
          string = line[:-1].split(';')
          out_string = ''
          for i in my_cols:
            out_string += string[i]+','
          output_f.write(out_string[:-1] + '\n')

src_folder = 'reports-main'
dest_folder = 'comb_reports'
process_files(src_folder, dest_folder)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv

import pandas as pd

user_answer = pd.read_csv(f'{dest_folder}/combined_data.csv')
correct_answer = pd.read_csv('data.csv')

--2025-03-09 15:02:24--  https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.github.com (gist.github.com)... 140.82.121.3
Connecting to gist.github.com (gist.github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv [following]
--2025-03-09 15:02:24--  https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 984 [text/plain]
Saving to: ‘data.csv.5’

data.csv.5          100%[===================>]     984  --.-KB/s    in 0s      

2025-03-09 15:02:24 (22.3 MB/s) - ‘data.csv.5’ saved [984/984]



In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!')

Поздравляем, Вы справились и успешно прошли все проверки!
